# READ-ME

This ipython notebook is used to run WaPOR-based Rapid WA+ analysis. 

## Flow-chart

## Table of contents

# 0. Start calculation
**Edit BASIN_NAME. Correct paths: MAIN_DIR, INPUT_DIR, PCKG_DIR**

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 23 12:36:04 2019

@author: ntr002
"""
import os
WORKING_DIR=r"D:\FAO\WA_Sheet1"
MAIN_DIR=os.path.join(WORKING_DIR,"Main")
INPUT_DIR=os.path.join(WORKING_DIR,"Input")
PCKG_DIR=os.path.join(WORKING_DIR,"WAPORWA")
### Import local package

os.chdir(PCKG_DIR) 
import WaPOR
import WA
from WA.pickle_basin import pickle_in, pickle_out
### Basin
BASIN_NAME='Litani'
INPUT_FOLDER=os.path.join(INPUT_DIR,BASIN_NAME)

**Input data**

In [2]:
#Global
GRaND_SHP=r"D:\FAO\WA_Sheet1\Input\Global\GRanD\GRanD_reservoirs_v1_1_clipped.shp"
WDPA_SHP=r"D:\FAO\WA_Sheet1\Input\Global\WDPA\WDPA_CatIandII_17countries.shp"
ThetaSat_TIF=r"D:\FAO\WA_Sheet1\Input\Global\HiHydroSoils\thetasat_topsoil.tif"

#Basin
BASIN_SHP=r"D:\FAO\WA_Sheet1\Input\Litani\Geo\Shapefile\Litani.shp"
GRACEMonthly_CSV=r"D:\FAO\Litani\WA_Data\GRACE\Litani_GRACE.csv"
unreserv_SHP=None
makereserv_SHP=None

## 0.1. Open latest pickle (saved metadata)

In [3]:
import glob
latest_pickle=glob.glob(os.path.join(MAIN_DIR,BASIN_NAME,'*.pickle'))[-1]
print(latest_pickle)
BASIN=pickle_in(latest_pickle)
end_month=BASIN['end_month']

D:\FAO\WA_Sheet1\Main\Litani\Info_20191106_11h33.pickle


In [ ]:
BASIN

# 1. Download WaPOR data
**Edit xmin, ymin, xmax, ymax**

In [37]:
WaPOR.API.version=2
catalog=WaPOR.API.getCatalog()
xmin,ymin,xmax,ymax=[35.22916666666592, 33.0999999999998, 36.399999999999224, 34.04999999999972]

# WaPOR.PCP_daily(INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2019-01-01',latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25])

# WaPOR.PCP_monthly(INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2019-01-01',latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25])

WaPOR.RET_monthly(INPUT_FOLDER,Startdate='2009-01-01',
                  Enddate='2019-01-01',latlim=[ymin-0.5,ymax+0.5],
                  lonlim=[xmin-0.5,xmax+0.5])

# WaPOR.AET_monthly(INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2019-01-01',latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

# WaPOR.LCC_yearly(INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2019-01-01',latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

# WaPOR.I_dekadal(INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2019-01-01',latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)


Loading WaPOR catalog...

Download monthly WaPOR Reference Evapotranspiration data for the period 2009-01-01 till 2019-01-01
Loading WaPOR catalog...
Progress: |██████████████████████████████████████████████████| 100.0% Complete


# 2. Initilize Basin metadata dictionary
**Edit: Basin metadata**

In [ ]:
BASIN={'Name':BASIN_NAME,
       'time_range':['2009-01-01','2018-12-31'],
       'end_month':'AUG',
       'Dir':os.path.join(MAIN_DIR,BASIN_NAME),       
        'geo_data':{
                   'basin':BASIN_SHP,
                   'unreserv':unreserv_SHP, #If GRaND reservoir map need to be adjusted
                   'makereserv':makereserv_SHP,#If GRaND reservoir map need to be adjusted
                      },
        'global_data':{                     
                     'grand':GRaND_SHP,
                     'wdpa':WDPA_SHP,
                     },
        'input_data':{
                      'yearly':{
                              'lcc':[r""+os.path.join(INPUT_FOLDER,'L2_LCC_A'),
                                     '-','Landcover Class'],
                              },
                      'monthly':{
                              'p':[r""+os.path.join(INPUT_FOLDER,'L1_PCP_M'),
                                   'mm/month','Precipitation'],
                              'et':[r""+os.path.join(INPUT_FOLDER,'L2_AETI_M'),
                                    'mm/month','Actual Evapotranspiration'],
                              'ret':[r""+os.path.join(INPUT_FOLDER,'L1_RET_M'),
                                    'mm/month','Reference Evapotranspiration']                                    
                              },  
                    'stat':{                                                          
                            'area':None,                 
                            'thetasat':ThetaSat_TIF
                            }                                                   
                              },
        'input_ts':{
                'dS':GRACEMonthly_CSV,
                
                    },
        'main_data':{
                'yearly':{},
                     'monthly':{},
                     'stat':{},
                     }                
                }

if not os.path.exists(BASIN['Dir']):
    os.makedirs(BASIN['Dir'])
pickle_out(BASIN)

# 3. Compute intermediate data
## 3.1. Calculate monthly Interception from L2_I_D
**If L2_I_M is not available via API**

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep 11 16:42:09 2019

@author: ntr002
"""
import os
os.chdir(PCKG_DIR)
import glob
import pandas as pd
from osgeo import gdal
import numpy as np
import osr
from WaPOR import GIS_functions as gis

#%%
#get list of dekadal rasters
Dekadal_I_folder=os.path.join(INPUT_FOLDER,'L2_I_D')
input_fhs=glob.glob(os.path.join(Dekadal_I_folder,'*.tif'))
input_fhs

#Get month dates
start_date=BASIN['time_range'][0]
end_date=BASIN['time_range'][1]
month_dates=pd.date_range(start_date,end_date,freq='M')

#Get df avail 
ds_code='L2_I_D' 
WaPOR.API.version=2
df_avail=WaPOR.API.getAvailData(ds_code,time_range='{0},{1}'.format(start_date,end_date))

#aggregate monthly
output_folder=os.path.join(INPUT_FOLDER,'L2_I_M')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(input_fhs[0])
for date in month_dates:
    month_fhs=[]
    for in_fh in input_fhs:
        raster_id=os.path.split(in_fh)[-1].split('.tif')[0][-9:]
        raster_info=df_avail.loc[df_avail['raster_id']==raster_id]
        timestr=raster_info['DEKAD'].iloc[0]
        year=int(timestr[0:4])
        month=int(timestr[5:7])
        if (year==date.year)&(month==date.month):
            month_fhs.append(in_fh)
    SumArray=np.zeros((ysize,xsize),dtype=np.float32)
    for fh in month_fhs:
        Array=gis.OpenAsArray(fh,nan_values=True)
        SumArray+=Array
    out_fh=os.path.join(output_folder,'L2_I_{:04d}{:02d}.tif'.format(date.year,date.month))    
    gis.CreateGeoTiff(out_fh, SumArray, driver, NDV, xsize, ysize, GeoT, Projection)
BASIN['input_data']['monthly']['i']=[r""+os.path.join(INPUT_FOLDER,'L2_I_M'),'mm/month','Interception']
pickle_out(BASIN)

## 3.2. Calculate number of rainy days per month

In [ ]:
"""
Adapted from watools (by T.Hessels)
"""
import calendar
import glob
import os
from WaPOR import GIS_functions as gis
import pandas as pd
import numpy as np
###
Data_Path_P=os.path.join(INPUT_FOLDER,'L1_PCP_E')
Startdate=BASIN['time_range'][0]
Enddate=BASIN['time_range'][1]

Data_Path_RD = os.path.join(BASIN['Dir'],'data', 'Rainy_Days')
if not os.path.exists(Data_Path_RD):
    os.mkdir(Data_Path_RD)
Dates = pd.date_range(Startdate, Enddate, freq ='MS')
os.chdir(Data_Path_P)

for Date in Dates:
    # Define the year and month and amount of days in month
    year = Date.year
    month = Date.month
    daysinmonth = calendar.monthrange(year, month)[1]

    # Set the third (time) dimension of array starting at 0
    i = 0

    # Find all files of that month
    files = glob.glob('*daily_%d.%02d.*.tif' %(year, month))

    # Check if the amount of files corresponds with the amount of days in month
    if len(files) is not daysinmonth:
        print('ERROR: Not all Rainfall days for month %d and year %d are downloaded'  %(month, year))

    # Loop over the days and store data in raster
    for File in files:
        dir_file = os.path.join(Data_Path_P, File)

        # Get array information and create empty numpy array for daily rainfall when looping the first file
        if File == files[0]:

            # Open geolocation info and define projection
            driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(dir_file)

            # Create empty array for the whole month
            P_Daily = np.zeros([daysinmonth,ysize, xsize])

        # Open data and put the data in 3D array
        Data = gis.OpenAsArray(dir_file,nan_values=True)

        # Remove the weird numbers
        Data[Data<0] = 0

        # Add the precipitation to the monthly cube
        P_Daily[i, :, :] = Data
        i += 1

    # Define a rainy day
    P_Daily[P_Daily > 0.201] = 1
    P_Daily[P_Daily != 1] = 0

    # Sum the amount of rainy days
    RD_one_month = np.nansum(P_Daily,0)

    # Define output name
    Outname = os.path.join(Data_Path_RD, 'Rainy_Days_NumOfDays_monthly_%d.%02d.01.tif' %(year, month))

    # Save tiff file
    gis.CreateGeoTiff(Outname, RD_one_month, driver, NDV, xsize, ysize, GeoT, Projection)
    
BASIN['input_data']['monthly']['nRD']=[Data_Path_RD,'days/month','Number of rainy days']
pickle_out(BASIN)

## 3.3. Reclassify WaPOR LCC to LUWA

In [1]:
"""
Created on Thu Aug  8 13:42:44 2019

@author: ntr002
WAPOR LCC to LUWA

Note: start Jupyter notebook with "conda activate env"
"""
import glob
from WA.LCC_to_LUWA import Rasterize_shape_basin,Adjust_GRaND_reservoir,Reclass_LCC_to_LUWA

### Get all yearly WaPOR LCC maps
LCC_fhs=glob.glob(os.path.join(BASIN['input_data']['yearly']['lcc'][0],'*.tif'))
WaPOR_LCC=LCC_fhs[0] #template

### Create Reservoir raster map for basin
Dir_out=os.path.join(BASIN['Dir'],'data','Reservoir')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
Basin_Reservoir_tif=os.path.join(Dir_out,'Reservoir_basin.tif')
Global_GRaND_Reservoir=BASIN['global_data']['grand']

# adjust reservoir
Resrv_to_Lake=BASIN['geo_data']['unreserv']
Lake_to_Reserv=BASIN['geo_data']['makereserv']  
if (Resrv_to_Lake!=None)&(Lake_to_Reserv!=None): #require 2 shapefiles to unreservoir and makereservoir           
    Adjust_GRaND_reservoir(Basin_Reservoir_tif,WaPOR_LCC,
                           Global_GRaND_Reservoir,Resrv_to_Lake,Lake_to_Reserv)    
else:                
    Rasterize_shape_basin(Global_GRaND_Reservoir,WaPOR_LCC,Basin_Reservoir_tif)
    
BASIN['geo_data']['reservoir']=Basin_Reservoir_tif

### Create Protected Area raster map for basin
Dir_out=os.path.join(BASIN['Dir'],'data','Protected')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
ProtectedArea_tif=os.path.join(Dir_out,'ProtectedArea_basin.tif')
WDPA=BASIN['global_data']['wdpa']
Rasterize_shape_basin(WDPA,WaPOR_LCC,ProtectedArea_tif)

### Reclassify LCC to LUWA       
Dir_out=os.path.join(BASIN['Dir'],'data','luwa')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
for fh in LCC_fhs:
    Reclass_LCC_to_LUWA(fh,Dir_out,ProtectedArea_tif,Basin_Reservoir_tif)

# Add LUWA folder to basin metadata
BASIN['input_data']['yearly']['lu']=[Dir_out,'-','LUWA Categories']
pickle_out(BASIN)              

NameError: name 'os' is not defined

# 4. Create netCDF for each variable

In [14]:
from WA.create_NC import main as make_netcdf
from WA.create_NC import _get_lats_lons 
import gdal
import tempfile

#Get inputs for create_NC
cutline=BASIN['geo_data']['basin']
Dir_out=os.path.join(BASIN['Dir'],'data','nc')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)    
template=glob.glob(os.path.join(BASIN['input_data']['yearly']['lcc'][0],'*.tif'))[0]          
    
### yearly maps
for key in BASIN['input_data']['yearly']:             
    if BASIN['input_data']['yearly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_yearly.nc')        
        dataset={key:[BASIN['input_data']['yearly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['yearly'][key][1],                                 
                        'quantity':BASIN['input_data']['yearly'][key][2],
                        'source': 'WaPOR', 'period':'year'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='year')
        if succes:
            BASIN['main_data']['yearly'][key]=nc_fn
            print('Finished {0}_yearly.nc'.format(key))
            
### monthly maps  
for key in BASIN['input_data']['monthly']:           
    if BASIN['input_data']['monthly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_monthly.nc')                
        dataset={key:[BASIN['input_data']['monthly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['monthly'][key][1],                                 
                        'quantity':BASIN['input_data']['monthly'][key][2],
                        'source': 'WaPOR', 'period':'month'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='month')
        if succes:
            BASIN['main_data']['monthly'][key]=nc_fn
            print('Finished {0}_monthly.nc'.format(key))
  
### static maps
lats, lons, optionsProj, optionsClip =_get_lats_lons(template,cutline)
Dir_out=os.path.join(BASIN['Dir'],'data','stat')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
for key in BASIN['input_data']['stat']:            
    if BASIN['input_data']['stat'][key] is not None:  
        basename= os.path.basename(BASIN['input_data']['stat'][key])
        outfn=os.path.join(Dir_out,basename)  
        temp_file = tempfile.NamedTemporaryFile(suffix='.tif').name               
        gdal.Warp(temp_file, BASIN['input_data']['stat'][key], options = optionsProj)
        gdal.Warp(outfn, temp_file, options = optionsClip)
        os.remove(temp_file)
        BASIN['main_data']['stat'][key]=outfn 
        print('Finished {0}'.format(basename))

pickle_out(BASIN)   

6.806621238863997e-16
-6.806621238863997e-16


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.38it/s]


Finished lcc_yearly.nc
6.806621238863997e-16
-6.806621238863997e-16


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.56it/s]


Finished lu_yearly.nc
6.806621238863997e-16
-6.806621238863997e-16


...

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [01:13<00:00,  1.17it/s]


Finished i_monthly.nc
6.806621238863997e-16
-6.806621238863997e-16


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:51<00:00,  2.25it/s]


Finished nRD_monthly.nc
6.806621238863997e-16
-6.806621238863997e-16
Finished thetasat_topsoil.tif


'D:\\FAO\\WA_Sheet1\\Main\\Litani\\Info_20191108_10h11.pickle'

## 4.1. Correct ET of water bodies by ET reference 
**If ETa of water bodies is underestimated**

In [ ]:
"""
Created on Mon Sep  2 19:45:28 2019

@author: sse
"""
import xarray as xr
import numpy as np
import pandas as pd
import os

def open_nc(nc,timechunk=1,chunksize=1000):
    dts=xr.open_dataset(nc)
    key=list(dts.keys())[0]
    var=dts[key].chunk({"time": timechunk, "latitude": chunksize, "longitude": chunksize}) #.ffill("time")
    return var,key

#%% replace Et of water bodies by ET reference (based on FAO's recommendation)
#lu
lcc,_=open_nc(BASIN['main_data']['yearly']['lcc'])
#ET
et,_ = open_nc(BASIN['main_data']['monthly']['et'])
#RET
ret,_= open_nc(BASIN['main_data']['monthly']['ret'])

lcc_dict = {
                20: 'Shrubland',
                30: 'Grassland', 
                41: 'Cropland, rainfed',
                42: 'Cropland, irrigated or under water management', 
                43: 'Cropland, fallow', 
                50: 'Built-up', 
                60: 'Bare / sparse vegetation', 
                70: 'Permament snow / ice', 
                80: 'Water bodies', 
                81: 'Temporary water bodies', 
                90: 'Shrub or herbaceous cover, flooded', 
                111: 'Tree cover: closed, evergreen needle-leaved', 
                112: 'Tree cover: closed, evergreen broadleaved', 
                114: 'Tree cover: closed, deciduous broadleaved', 
                115: 'Tree cover: closed, mixed type', 
                116: 'Tree cover: closed, unknown type', 
                121: 'Tree cover: open, evergreen needle-leaved', 
                122: 'Tree cover: open, evergreen broadleaved', 
                123: 'Tree cover: open, deciduous needle-leaved', 
                124: 'Tree cover: open, deciduous broadleaved', 
                125: 'Tree cover: open, mixed type',
                126: 'Tree cover: open, unknown type', 
                200: 'Sea water'
          }
##
et_lcc_t = []

for i in range(len(lcc.time)): 
    t1 = i*12
    t2 = (i+1)*12
    lcc_t = lcc.isel(time = i)
    et_m_t = []
    for t in range(t1,t2):
        et_t = et.isel(time = t)  
        ret_t = ret.isel(time = t)  
        et_updated_t=et_t.where((lcc_t!=80)&(lcc_t!=200),ret_t)
        #et_updated_t.assign_coords({'time': et_t.time})
        et_updated_t = et_updated_t.assign_coords(time=et_t.time)
        et_updated_t = et_updated_t.expand_dims('time')
        et_m_t.append(et_updated_t)
    et_m = xr.concat(et_m_t, dim = 'time')
    et_lcc_t.append(et_m)
et_modified= xr.concat(et_lcc_t, dim='time')    
et_modified=et_modified.chunk({"latitude":-1,"longitude":-1}).to_dataset()

### save corrected ETa as netCDF
nc_fn=r"et_corrected_monthly.nc"
outfolder = os.path.join(BASIN['Dir'],'data','nc')
nc_path=os.path.join(outfolder,nc_fn)
et_modified.to_netcdf(nc_path,encoding={"Actual Evapotranspiration":{'zlib':True}})
BASIN['main_data']['monthly']['et']=nc_path
pickle_out(BASIN)

# 5. Pixel-analysis: Compute ET$_{incr}$ and ET$_{rain}$

## 5.1. Read Input from Basin metadata

In [12]:
## parameters
start_year=2009
f_perc=1 # percolation factor
f_Smax=0.9
cf =  20 #f_Ssat soil mositure correction factor to componsate the variation in filling up and drying in a month
    
## input data
smsat_file = BASIN['main_data']['stat']['key']
## read nc files    
p_in = BASIN['main_data']['monthly']['p']
e_in = BASIN['main_data']['monthly']['et']
i_in = BASIN['main_data']['monthly']['i']
rd_in = BASIN['main_data']['monthly']['nRD']
lu_in = BASIN['main_data']['yearly']['lcc']

## 5.2. Run Monthly Soil moisture Balance Model

### Start a status Dashboard of dasked array

In [ ]:
from dask.distributed import Client
client=Client()
client

### Main scripts

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 18 08:22:17 2019

@author: Solomon Seyoum
"""

import os
import numpy as np
import gdal
import xarray as xr

#%%
def open_nc(nc,chunksize=1000):
    dts=xr.open_dataset(nc)
    key=list(dts.keys())[0]
    var=dts[key].chunk({"time": 1, "latitude": chunksize, "longitude": chunksize}) #.ffill("time")
    return var,key

def SM_bucket(P,ETa,I,SMmax,SMgt_1,SMincrt_1,f_consumed):
    SMt_1=SMgt_1+SMincrt_1
    ETr=np.where(I>0,ETa-I, ETa)
    SMtemp=SMgt_1+np.maximum(P-I,P*0)
    SMg=np.where(SMtemp-ETr>0,SMtemp-ETr,P*0)
    ETincr=np.where(SMtemp-ETr>0,P*0,ETr-SMtemp)
    Qsupply=np.where(ETincr>SMincrt_1,(ETincr-SMincrt_1)/f_consumed, P*0) 
            
    #SMincr=np.where(ETincr>SMincrt_1,SMincrt_1+Qsupply-ETincr,SMincrt_1-ETincr)
    SMincr=SMincrt_1+Qsupply-ETincr
    SMincr=np.where(SMg+SMincr>SMmax,SMincr-SMincr/(SMg+SMincr)*((SMg+SMincr)-SMmax),SMincr)     
    SMg=np.where(SMg+SMincr>SMmax,(SMg-SMg/(SMg+SMincr)*((SMg+SMincr)-SMmax)),SMg)
    
    #SMincr=np.where(SMg+SMincr>SMmax,SMincr-((SMg+SMincr)-SMmax),SMincr)
    SM=np.where(SMg+SMincr>SMmax,SMmax,SMg+SMincr)
    dsm=SM-SMt_1
    ETg=ETa-ETincr
    return SMg,SMincr,SM,dsm,Qsupply,ETincr,ETg


def get_rootdepth(version = '1.0'):
    
    lcc_code = dict()
    lcc_code['1.0'] = {
       'Shrubland':20,
       'Grassland':30,
       'Rainfed Cropland':41,
       'Irrigated Cropland':42,
       'Fallow Cropland':43,
       'Built-up':50,
       'Bare/sparse vegetation':60,
       'Permanent snow/ ice':70,
       'Water bodies':80,
       'Temporary water bodies':81,
       'Shrub or herbaceous cover, flooded':90,     
       'Tree cover: closed, evergreen needle-leaved':111,
       'Tree cover: closed, evergreen broad-leaved':112, 
       'Tree cover: closed, deciduous broad-leaved':114, #
       'Tree cover: closed, mixed type':115, #
       'Tree cover: closed, unknown type':116, #
       'Tree cover: open, evergreen needle-leaved':121,#
       'Tree cover: open, evergreen broad-leaved':122, #
       'Tree cover: open, deciduous needle-leaved':123, #
       'Tree cover: open, deciduous broad-leaved':124, #
       'Tree cover: open, mixed type':125, #
       'Tree cover: open, unknown type':126, #     
       'Seawater':200, #       
       }
    
    root_depth = dict()
    '''
    based on Global estimation of effective plant rooting depth: 
    Implications for hydrological modeling by Yang et al (2016)
    '''
    root_depth['1.0'] = {
       'Shrubland':370,
       'Grassland':510,
       'Rainfed Cropland':550,
       'Irrigated Cropland':550,
       'Fallow Cropland':550,
       'Built-up':370,
       'Bare/sparse vegetation':370,
       'Permanent snow/ ice':0,
       'Water bodies':0,
       'Temporary water bodies':0,
       'Shrub or herbaceous cover, flooded':0,     
       'Tree cover: closed, evergreen needle-leaved':1800,
       'Tree cover: closed, evergreen broad-leaved':3140, 
       'Tree cover: closed, deciduous broad-leaved':1070, #
       'Tree cover: closed, mixed type':2000, #
       'Tree cover: closed, unknown type':2000, #
       'Tree cover: open, evergreen needle-leaved':1800,#
       'Tree cover: open, evergreen broad-leaved':3140, #
       'Tree cover: open, deciduous needle-leaved':1070, #
       'Tree cover: open, deciduous broad-leaved':1070, #
       'Tree cover: open, mixed type':2000, #
       'Tree cover: open, unknown type':2000, #     
       'Seawater':0, #       
    }
    
    return lcc_code[version], root_depth[version]

def root_dpeth(lu):
    rootdepth=np.copy(lu)
    lu_categories, root_depth = get_rootdepth(version = '1.0')
    for key in root_depth.keys():
        mask = np.logical_or.reduce([lu == lu_categories[key]])
        rd = root_depth[key]
        rootdepth[mask] = rd
    return rootdepth 


def get_fractions(version = '1.0'):
    consumed_fractions = dict()
    
    consumed_fractions['1.0'] = {
       'Shrubland':1.00,
       'Grassland':1.00,
       'Rainfed Cropland':1.00,
       'Irrigated Cropland':0.80,
       'Fallow Cropland':1.00,
       'Built-up':1.00,
       'Bare/sparse vegetation':1.00,
       'Permanent snow/ ice':1.00,
       'Water bodies':1.00,
       'Temporary water bodies':1.00,
       'Shrub or herbaceous cover, flooded':1.00,     
       'Tree cover: closed, evergreen needle-leaved':1.00,
       'Tree cover: closed, evergreen broad-leaved':1.00, 
       'Tree cover: closed, deciduous broad-leaved':1.00, #
       'Tree cover: closed, mixed type':1.00, #
       'Tree cover: closed, unknown type':1.00, #
       'Tree cover: open, evergreen needle-leaved':1.00,#
       'Tree cover: open, evergreen broad-leaved':1.00, #
       'Tree cover: open, deciduous needle-leaved':1.00, #
       'Tree cover: open, deciduous broad-leaved':1.00, #
       'Tree cover: open, mixed type':1.00, #
       'Tree cover: open, unknown type':1.00, #     
       'Seawater':1.00, #       
#            'Forests':              1.00,
#            'Shrubland':            1.00,
#            'Rainfed Crops':        1.00,
#            'Forest Plantations':   1.00,
#            'Natural Water Bodies': 1.00,
#            'Wetlands':             1.00,
#            'Natural Grasslands':   1.00,
#            'Other (Non-Manmade)':  1.00,
#            'Irrigated crops':      0.80,
#            'Managed water bodies': 1.00,
#            'Other':                1.00,
#            'Residential':          1.00,
#            'Greenhouses':          0.95,
#            'Aquaculture':          1.00
    }
    
    return consumed_fractions[version]

def Consumed_fraction(lu):
    f_consumed=np.copy(lu)
    consumed_fractions = get_fractions(version = '1.0')
    lu_categories, root_depth = get_rootdepth(version = '1.0')
    for key in consumed_fractions.keys():
        mask = np.logical_or.reduce([lu == lu_categories[key]])
        consumed_fraction = consumed_fractions[key]
        f_consumed[mask] = consumed_fraction
    return f_consumed 

#%%

In [17]:
%%time
Pt,key=open_nc(p_in,chunksize=1000)
E,key=open_nc(e_in,chunksize=1000)
Int,key=open_nc(i_in,chunksize=1000)
nRD,key=open_nc(rd_in,chunksize=1000)
LU,key=open_nc(lu_in,chunksize=1000)

##read saturation file
ds = gdal.Open(smsat_file)
thetasat = ds.ReadAsArray()

nrd = nRD.where(nRD!=0,1)
 
SMg=E[0]*0
SMincr=E[0]*0

for j in range(len(LU.time)): 
    t1 = j*12
    t2 = (j+1)*12    
    etb = np.zeros((12,E[0].shape[0],E[0].shape[1]))
    etg = np.zeros((12,E[0].shape[0],E[0].shape[1]))
    lu = LU.isel(time=j).values
    Rd = root_dpeth(lu)
    SMmax=thetasat*Rd    
    f_consumed = Consumed_fraction(lu)    
    for t in range(t1,t2):
        print('time: ', t)
        if t==0:
            SMgt_1=E[0]*0
            Qsupply=E[0]*0
            SMincrt_1=E[0]*0
        else:
            SMgt_1=SMg 
            SMincrt_1=SMincr         
        p = Pt.isel(time=t).values
        e = E.isel(time=t).values
        i = Int.isel(time=t).values
        NRD = nrd.isel(time=t).values
        P = p/NRD
        ETa = e/NRD
        I = i/NRD
        
        ### Step 1: Soil moisture
        SMg,SMincr,SM,dsm,Qsupply,ETincr,ETg=SM_bucket(P,ETa,I,SMmax,SMgt_1,SMincrt_1,f_consumed) 
    
        ### Step 3: Percolation        
        perc_fromSM=np.where(SM>0.6*SMmax,(1-np.exp(-f_perc/SM)),P*0)
        SM = SM - perc_fromSM
        SMt_1= SMgt_1+SMincrt_1
        dsm = SM-SMt_1
        
        ### Step 5: Store monthly data of the year
        k = int(t-(j*12))
        etb[k,:,:] = ETincr*NRD
        etg[k,:,:] = ETg*NRD
        
        del p
        del ETa
        del I
        del NRD
        del SMgt_1 
        del Qsupply 
        del SMincrt_1
        
    year = start_year+j    
#    E['etb'] = (('time','latitude', 'longitude'), etb)
#    E['etg'] = (('time','latitude', 'longitude'), etg)
    time_ds = E.time[t1:t2]
    ds = xr.Dataset({})
    ds['etb'] = (('time','latitude', 'longitude'), etb)
    ds['etg'] = (('time','latitude', 'longitude'), etg)
    ds = ds.assign_coords(time = time_ds, latitude=E.latitude,longitude=E.longitude )
    
    ##green ET datase
    et_g = ds.etg
    attrs={"units":"mm/month", "source": "FAO WaPOR", "quantity":"Rainfall_ET_M"}
    et_g.assign_attrs(attrs)
    et_g.name = "Rainfall_ET_M"
    et_g_dts=et_g.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    outfolder=os.path.join(Basin['Dir'],'data','nc','etg')
    nc_fn=r'et_g_monthly_'+str(year)+'.nc'
    nc_path=os.path.join(outfolder,nc_fn)
    comp = dict(zlib=True, complevel=9, least_significant_digit=3)
    encoding = {"Rainfall_ET_M": comp}
    et_g_dts.to_netcdf(nc_path,encoding=encoding)
    
    ##Blue ET datase
    et_b = ds.etb
    attrs={"units":"mm/month", "source": "FAO WaPOR", "quantity":"Increamental_ET_M"}
    et_b.assign_attrs(attrs)
    et_b.name = "Increamental_ET_M"
    et_b_dts=et_b.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    outfolder=os.path.join(Basin['Dir'],'data','nc','etb')
    nc_fn=r'et_b_monthly_'+str(year)+'.nc'
    nc_path=os.path.join(outfolder,nc_fn)
    et_b_dts.to_netcdf(nc_path,encoding={"Increamental_ET_M":{'zlib':True}})
    
    del etb
    del etg

time:  0
time:  1
time:  2
time:  3
time:  4
time:  5
time:  6
time:  7
time:  8
time:  9
time:  10
time:  11
time:  12
time:  13
time:  14
time:  15
time:  16
time:  17
time:  18
time:  19
time:  20
time:  21
time:  22
time:  23
time:  24
time:  25
time:  26
time:  27
time:  28
time:  29
time:  30
time:  31
time:  32
time:  33
time:  34
time:  35
time:  36
time:  37
time:  38
time:  39
time:  40
time:  41
time:  42
time:  43
time:  44
time:  45
time:  46
time:  47
time:  48
time:  49
time:  50
time:  51
time:  52
time:  53
time:  54
time:  55
time:  56
time:  57
time:  58
time:  59
time:  60
time:  61
time:  62
time:  63
time:  64
time:  65
time:  66
time:  67
time:  68
time:  69
time:  70
time:  71
time:  72
time:  73
time:  74
time:  75
time:  76
time:  77
time:  78
time:  79
time:  80
time:  81
time:  82
time:  83
time:  84
time:  85
time:  86
time:  87
time:  88
time:  89
time:  90
time:  91
time:  92
time:  93
time:  94
time:  95
time:  96
time:  97
time:  98
time:  99
time:  100

## 5.3. Merge monthly ET$_{rain}$ and ET$_{incr}$ netcdf files

In [18]:
nc_folder=r"D:\FAO\WA_Sheet1\Main\Litani\data\nc\etg"
fhs=glob.glob(os.path.join(nc_folder,'*.nc'))
fhs

['D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2009.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2010.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2011.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2012.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2013.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2014.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2015.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2016.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2017.nc',
 'D:\\FAO\\WA_Sheet1\\Main\\Litani\\data\\nc\\etg\\et_g_monthly_2018.nc']

In [19]:
arr_list=[]
for fh in fhs:
    arr,key=open_nc(fh)
    arr_list.append(arr)
ETg=xr.concat(arr_list, dim='time')
ETg

<xarray.DataArray 'Rainfall_ET_M' (time: 120, latitude: 958, longitude: 1181)>
dask.array<shape=(120, 958, 1181), dtype=float64, chunksize=(1, 958, 1000)>
Coordinates:
  * latitude   (latitude) float32 34.049603 34.04861 ... 33.10119 33.100197
  * longitude  (longitude) float32 35.229168 35.23016 ... 36.39881 36.399803
  * time       (time) datetime64[ns] 2009-01-01 2009-02-01 ... 2018-12-01

In [20]:
attrs={"units":"mm/month", "source": "FAO WaPOR", "quantity":"Rainfall_ET_M"}
ETg.assign_attrs(attrs)
ETg_dts=ETg.chunk({"latitude":-1,"longitude":-1}).to_dataset()
outfolder=os.path.join(Basin['Dir'],'data','nc')
nc_fn=r'etg_monthly.nc'
nc_path=os.path.join(outfolder,nc_fn)
comp = dict(zlib=True, complevel=9, least_significant_digit=3)
encoding = {"Rainfall_ET_M": comp}
ETg_dts.to_netcdf(nc_path,encoding=encoding)

In [21]:
nc_folder=r"D:\FAO\WA_Sheet1\Main\Litani\data\nc\etb"
fhs=glob.glob(os.path.join(nc_folder,'*.nc'))
arr_list=[]
for fh in fhs:
    arr,key=open_nc(fh)
    arr_list.append(arr)
ETb=xr.concat(arr_list, dim='time')
attrs={"units":"mm/month", "source": "FAO WaPOR", "quantity":"Incremental_ET_M"}
ETb.assign_attrs(attrs)
ETb_dts=ETb.chunk({"latitude":-1,"longitude":-1}).to_dataset()
outfolder=os.path.join(Basin['Dir'],'data','nc')
nc_fn=r'etb_monthly.nc'
nc_path=os.path.join(outfolder,nc_fn)
comp = dict(zlib=True, complevel=9, least_significant_digit=3)
encoding = {"Increamental_ET_M": comp}
ETb_dts.to_netcdf(nc_path,encoding=encoding)

In [22]:
Basin['main_data']['monthly']['etg']=r"D:\FAO\WA_Sheet1\Main\Litani\data\nc\etg_monthly.nc"
Basin['main_data']['monthly']['etb']=r"D:\FAO\WA_Sheet1\Main\Litani\data\nc\etb_monthly.nc"
pickle_out(Basin)

'D:\\FAO\\WA_Sheet1\\Main\\Litani\\Info_20191106_11h33.pickle'

# 6. Aggregate annual value by hydrological year
**Variables: ET (water bodies corrected with ETref), ETincr, ETrain**

In [ ]:
time='A-{0}'.format(BASIN['end_month'])
for key in ['p','et','etb','etg']:
    nc=Basin['main_data']['monthly'][key]
    var,name=open_nc(nc)
    var_y=var.resample(time=time).sum(dim='time',skipna=False)
    outfolder=os.path.join(Basin['Dir'],'data','nc')  
    attrs={"units":"mm/year", "source": "Hydrolological year Sep-Aug", "quantity":name}
    var_y.assign_attrs(attrs)
    var_y.name = name
    var_y_dts=var_y.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    nc_fn="{0}_hyearly1.nc".format(key)
    nc_path=os.path.join(outfolder,nc_fn)
    var_y_dts.to_netcdf(nc_path,encoding={name:{'zlib':True}})
    Basin['main_data']['yearly'][key]=nc_path

# 7. Calculate fluxes volumes table for Sheet 1

In [ ]:
#%% Area mask
# from WA import GIS_functions as gis

# # get template from LU map
# template=glob.glob(os.path.join(Basin['input_data']['yearly']['lu'][0],"*.tif"))[0]
# driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(template)
# # calculate area per pixel
# area_map=gis.MapPixelAreakm(template)
# # save area map as tif
# Basin['input_data']['stat']['area']=os.path.join(Basin['Dir'],'data','stat','Area_km.tif')
# gis.CreateGeoTiff(Basin['input_data']['stat']['area'],area_map,driver, NDV, xsize, ysize, GeoT, Projection)

# # create area mask
# shape=Basin['geo_data']['basin']
# area=Basin['input_data']['stat']['area']
# Basin['main_data']['stat']['area']=os.path.join(Basin['Dir'],'data','stat','Basin_Area_km.tif')
# gis.Clip_shapefile(area,shape,Basin['main_data']['stat']['area'])


### Get Region of Interest

In [ ]:
from WA import GIS_functions as gis
import xarray as xr
area_fh=Basin['main_data']['stat']['area']
# area_fh=Basin['main_data']['subbasin']['Zarqa']
area=gis.OpenAsArray(area_fh,nan_values=True)

#### Calculate P, ET, ETb, ETg

In [ ]:
ts_all=[] #all timeseries
for key in ['p','et','etg','etb']:
    nc=Basin['main_data']['yearly'][key]
    dts=xr.open_dataset(nc)
    var_name=list(dts.keys())[0]
    var=dts[var_name].chunk({"time": 1, "latitude": 1000, "longitude": 1000}).ffill("time")
    #attrs=dts[var_name].attrs
    Volume=var*area
    #attrs['units']='TCM/year'
    #attrs['quantity']='Precipitation (TCM/year)'
    #Volume.assign_attrs(attrs)
    ts=Volume.sum(dim=['latitude','longitude']).to_dataframe()
    ts.index=[y.year for y in ts.index]
    ts_all.append(ts)

#### Calculate ETb, ETg per LU category

In [ ]:
LU_dts=xr.open_dataset(Basin['main_data']['yearly']['lu'])
ETg_dts=xr.open_dataset(Basin['main_data']['yearly']['etg'])
ETb_dts=xr.open_dataset(Basin['main_data']['yearly']['etb'])

ETg=ETg_dts["Rainfall_ET_M"].chunk({"time": 1, "latitude": 1000, "longitude": 1000}).ffill("time")
ETb=ETb_dts["Increamental_ET_M"].chunk({"time": 1, "latitude": 1000, "longitude": 1000}).ffill("time")
LU=LU_dts["LUWA Categories"].chunk({"time": 1, "latitude": 1000, "longitude": 1000}).ffill("time")

### Different year date to year 
LU=LU.groupby('time.year').first(skipna=False)
ETb=area*ETb.groupby('time.year').first(skipna=False)
ETg=area*ETg.groupby('time.year').first(skipna=False)
### average per LU
from WA.average_by_LU import Total_perLU
ts_ETb=Total_perLU(ETb,LU)
ts_ETg=Total_perLU(ETg,LU)

ts_all.append(ts_ETb)
ts_all.append(ts_ETg)

#### Read GRACE dS

In [ ]:
import pandas as pd
import numpy as np
df_dS_y=pd.read_csv(r"D:\FAO\WA_Sheet1\Main\Jordan\data\GRACE\dS_y_cmi.csv",sep=';',index_col=0)
# df_dS_y=pd.read_csv(r"D:\FAO\WA_Sheet1\Main\Jordan\data\GRACE\dS_y_Yarmouk_cmi.csv",sep=';',index_col=0)
# df_dS_y=pd.read_csv(r"D:\FAO\WA_Sheet1\Main\Jordan\data\GRACE\dS_y_Zarqa_cmi.csv",sep=';',index_col=0)
Area_skm=np.nansum(area)
df_grace_ds=df_dS_y*Area_skm
#df_grace_ds.index=[y.year for y in df_grace_ds.index]
df_grace_ds=df_grace_ds.rename(columns={'dS [mm]':'GRACE_dS'})
df_grace_ds=df_grace_ds.rename(columns={'Equivalent Water Height [mm]':'GRACE_dS'})
ts_all.append(df_grace_ds)
for i in range(len(ts_all)):
    if i ==0:
        df=ts_all[i]        
    else:
        df=pd.merge(df,ts_all[i],left_index=True,right_index=True,how='outer')

#### Read Qout 

In [ ]:
import datetime
import matplotlib.pyplot as plt
df_q_out=pd.read_csv(r"D:\FAO\WA_Sheet1\Main\Jordan\data\Q_NWC.csv",sep=';',index_col=0)
df_q_out.index=[datetime.datetime.strptime(y,'%d/%m/%Y') for y in df_q_out.index]

df_q_out_y=df_q_out.resample('A-{0}'.format(end_month)).sum() #mean()
df_q_out_y.index=[y.year for y in df_q_out_y.index]

df=pd.merge(df,df_q_out_y,left_index=True,right_index=True,how='outer')

df['dS']=(df['Precipitation']-df['Actual Evapotranspiration']-df['Q_NWC'])
df['Error']=df['dS']-df['GRACE_dS']

plt.plot(df['Error'],label='error')
plt.plot(df['Q_NWC'],label='nwc')
plt.plot(df['GRACE_dS'],label='grace')
plt.legend()
plt.show()

In [ ]:
#%% Read Qout subbasins
import datetime
df_q_out=pd.read_csv(r"D:\FAO\Jordan\Obs_data\Flows\ADASIYIA.csv",sep=';',index_col=0,skiprows=1,usecols=[0,4])
df_q_out.index=[datetime.datetime.strptime(y,'%d/%m/%Y %H:%M') for y in df_q_out.index]
##
df_q_out=pd.read_csv(r"D:\FAO\Jordan\Obs_data\Flows\NEWJERASHBRIDGE.csv",sep=';',index_col=0,skiprows=1,usecols=[0,4])
df_q_out.index=[datetime.datetime.strptime(y,'%Y-%m-%d %H:%M:%S') for y in df_q_out.index]
df_q_out=df_q_out.rename(columns={'data':'Qout'})

df_q_out=df_q_out.replace(-9999,np.nan)
df_q_out_y=df_q_out.resample('A-{0}'.format(end_month)).sum() #mean()
df_q_out_y.index=[y.year for y in df_q_out_y.index]
df_q_out_y=df_q_out_y.Qout*1000000 #from km3 to TCM
df=pd.merge(df,df_q_out_y,left_index=True,right_index=True,how='outer')
df=df.dropna()
df['dS']=(df['Precipitation']-df['Actual Evapotranspiration']-df['Qout'])
df['Error']=df['dS']-df['GRACE_dS']

plt.plot(df['Error'],label='error')
plt.plot(df['Qout'],label='Qout')
plt.plot(df['GRACE_dS'],label='grace')
plt.legend()
df.to_csv(r'D:\FAO\WA_Sheet1\Main\Jordan\subbasins\df_Zarqa.csv',sep=';')


# 8. Sheet 1 generators

### Write fluxes volume table

In [ ]:
df
# df.to_csv(r"D:\FAO\WA_Sheet1\Main\Jordan\df_all_191029.csv",sep=';')

### Read fluxes volume table

In [ ]:
df=pd.read_csv(r"D:\FAO\WA_Sheet1\Main\Jordan\df_all_191029.csv",sep=';',index_col=0)
df

## 8.1 Create Yearly Sheet 1 csv

In [ ]:
df=df/100000 #convert from TCM to 0.1 BCM
csv_template=r"D:\FAO\WA_Sheet1\WAPORWA\WA\csv\Sample_sheet1.csv"
csv_folder=os.path.join(Basin['Dir'],'data','csv')
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)
df_template=pd.read_csv(csv_template,sep=";")
csv_fhs=[]
for year in df.index[1:-1]:
    df_year=df_template.copy()
    df_year['VALUE']=0.0
    df_year.loc[0,'VALUE']=df.loc[year,'Precipitation']
#    if df.loc[year,'dS']>0:
#        df_year.loc[11,'VALUE']=df.loc[year,'dS']
#    else:
#        df_year.loc[10,'VALUE']=abs(df.loc[year,'dS'])
    df_year.loc[10,'VALUE']=-df.loc[year,'dS']
    df_year.loc[12,'VALUE']=df.loc[year,'Rainfall_ET_M-Protected Landuse']
    df_year.loc[13,'VALUE']=df.loc[year,'Rainfall_ET_M-Utilized Landuse']
    df_year.loc[14,'VALUE']=df.loc[year,'Rainfall_ET_M-Modified Landuse']
    df_year.loc[15,'VALUE']=df.loc[year,'Rainfall_ET_M-Managed Water Use']
    df_year.loc[16,'VALUE']=df.loc[year,'Increamental_ET_M-Protected Landuse']
    df_year.loc[17,'VALUE']=df.loc[year,'Increamental_ET_M-Utilized Landuse']
    df_year.loc[18,'VALUE']=df.loc[year,'Increamental_ET_M-Modified Landuse']
    df_year.loc[19,'VALUE']=df.loc[year,'Increamental_ET_M-Managed Water Use']
    df_year.loc[20,'VALUE']=df.loc[year,'Increamental_ET_M-Managed Water Use']
    df_year.loc[21,'VALUE']=df.loc[year,'Increamental_ET_M']-df.loc[year,'Increamental_ET_M-Managed Water Use']
    df_year.loc[26,'VALUE']=df.loc[year,'Q_NWC']
    outcsv=os.path.join(csv_folder,'Sheet1_{0}.csv'.format(int(year)))
    df_year.to_csv(outcsv,sep=";",index=False)
    csv_fhs.append(outcsv)

## 8.2 Create yearly Sheet 1 Png

In [ ]:
#%% create sheet 1 png
csv_folder=os.path.join(Basin['Dir'],'data','csv')
csv_fhs=glob.glob(os.path.join(csv_folder,'Sheet1_*.csv'))
svg_template=r"D:\FAO\WA_Sheet1\WAPORWA\WA\svg\sheet_1_v2_0.1Bm3yr-1.svg"
png_folder=os.path.join(Basin['Dir'],'data','sheet1')
if not os.path.exists(png_folder):
    os.makedirs(png_folder)
from WA.sheet1 import create_sheet1
for csv in csv_fhs:
    period=os.path.basename(csv).split('_')[1].split('.')[0] 
    output=os.path.join(png_folder,os.path.basename(csv).replace('csv','png'))
    create_sheet1('Jordan', period, '0.1 BCM/year', csv, output, template=svg_template)



## 8.3 Create Mean annual sheets

In [ ]:
#%% Average sheet 1
def average_sheet(sheet_folder,sheet_number,period):    
    """
    Calculate average of yearly WA+ sheets in csv format
    Applicable for Sheet2, 3a, 4, 5, 6
    
    Parameters
    -------------
    sheet_folder: str
        folder of all sheet csv
    sheet_number: int
        number from 1 to 6
    period: list
        period to calculate average [yyyy,yyyy]
    unit: str
        unit of values in average csv file (Mm3 or km3) default is Mm3
        
    Returns
    -------------
    outname: str
        path to average csv
    """
    
    sheet_indexcol = {1: (0,1,2),
                   2: (0,1),
                   3: (0,1,2,3,4),
                   4: (0),
                   5: (0,1,3),
                   6: (0,1)}
    year_ls = pd.date_range(start='1/1/{0}'.format(period[0]),end='12/31/{0}'.format(period[1]),freq='Y')
    
    df_avg=0  
    for year in year_ls:
        sheet_csv = glob.glob(os.path.join(sheet_folder,'*{0}.csv'.format(str(year.year))))[0]
        df=pd.read_csv(sheet_csv,sep=';',index_col=sheet_indexcol[sheet_number],na_values='nan')
        df_avg+=df
    df_avg=df_avg/len(year_ls)
    
    outname=os.path.join(sheet_folder,'sheet{0}_{1}-{2}.csv'.format(sheet_number,period[0],period[1]))
    df_avg.to_csv(outname,sep=';',na_rep='nan')
    return outname

csv=average_sheet(r'D:\FAO\WA_Sheet1\Main\Jordan\data\csv',1,[2010,2018])
#%%
csv=r"D:\FAO\WA_Sheet1\Main\Jordan\data\csv\sheet1_2010-2018.csv"
output=os.path.join(png_folder,os.path.basename(csv).replace('csv','png'))
create_sheet1('Jordan', '2010-2018', '0.1 BCM/year', csv, output, template=svg_template)